Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы

In [ ]:
pip install wget


In [ ]:
pip install stop_words

In [ ]:
pip install annoy

In [ ]:
pip install pymorphy2

In [ ]:
import pandas as pd
import numpy as np
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

In [ ]:
RANDOM_STATE = 42
NUM_THREADS = 12

Загрузка и подготовка данных

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/NLP/3 урок/gazeta.csv', usecols=['text', 'title'])
df.head()

In [ ]:
df.shape

In [ ]:
# Возьму 10к случайных текстов, полный объем считает очень долго
data = df.sample(10000)
data.head()

In [ ]:
data.shape

Препроцессинг текста

In [ ]:
assert True

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
%%time

with Pool(NUM_THREADS) as p:
    data['processed_text'] = list(tqdm(p.imap(preprocess_txt, data['text']), total=len(data)))

In [ ]:
data.head(3)

Создание модели

In [ ]:
# Создание модели Word2Vec
%%time
modelW2V = Word2Vec(data['processed_text'], size=300, seed=RANDOM_STATE, window=3, workers=NUM_THREADS, min_count=2)

In [ ]:
# Создание модели FastText
%%time
modelFT = FastText(data['processed_text'], size=300, min_count=2, window=3, seed=RANDOM_STATE, workers=NUM_THREADS)

In [ ]:
%%time
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

counter = 0
index_map = {}

for line in data['text']:
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line
    text = preprocess_txt(line)
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in text:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

In [ ]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

Проверка работы модели

In [ ]:
TEXT = "чемпионат мира по футболу"

In [ ]:
get_response(TEXT, w2v_index, modelW2V, index_map)

In [ ]:
get_response(TEXT, ft_index, modelFT, index_map)